In [ ]:
from google.colab import files
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import zipfile
import tensorflow as tf


In [ ]:

# Upload a zip file containing your train and test folders
uploaded = files.upload()

# Specify the name of the uploaded zip file
zip_file_name = next(iter(uploaded))

# Create a directory to extract the contents
extracted_dir = 'extracted_images'
os.makedirs(extracted_dir, exist_ok=True)



Saving New folder.zip to New folder.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Define paths to train and test folders
train_folder = os.path.join(extracted_dir, 'New folder', 'Train')
test_folder = os.path.join(extracted_dir, 'New folder', 'Test')

# List the contents of the train and test folders
print(f'Contents of {train_folder}: {os.listdir(train_folder)}')
print(f'Contents of {test_folder}: {os.listdir(test_folder)}')

# Create data generators
batch_size = 64



Contents of extracted_images/New folder/Train: ['1', '2']
Contents of extracted_images/New folder/Test: ['1', '2']


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)



Found 8724 images belonging to 2 classes.
Found 2908 images belonging to 2 classes.


In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = True

# Create your custom model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/25
137/137 [==============================] - 229s 1s/step - loss: 0.3751 - accuracy: 0.8513 - val_loss: 2.3215 - val_accuracy: 0.5158
Epoch 2/25
137/137 [==============================] - 168s 1s/step - loss: 0.1723 - accuracy: 0.9380 - val_loss: 1.4148 - val_accuracy: 0.5158
Epoch 3/25
137/137 [==============================] - 169s 1s/step - loss: 0.1221 - accuracy: 0.9555 - val_loss: 1.8957 - val_accuracy: 0.5158
Epoch 4/25
137/137 [==============================] - 168s 1s/step - loss: 0.0877 - accuracy: 0.9700 - val_loss: 2.1312 - val_accuracy: 0.5158


KeyboardInterrupt: ignored

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
# Generate predictions
y_true = validation_generator.classes
y_pred_probs = model.predict(validation_generator)

class_indices = validation_generator.class_indices
print("Class Indices:", class_indices)

# Reorder the predictions based on class indices
y_pred_reordered = np.argmax(y_pred_probs, axis=1)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_reordered)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_true, y_pred_reordered)
print("\nClassification Report:")
print(class_report)

In [ ]:
# Save the entire model to a file
model.save('your_model.h5')

# Download the saved model file
files.download('your_model.h5')